## Repeated MCX simulation Stage 4.2


In [1]:
! pip install pmcx
! pip install numpy pmcx jdata bjdata matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import pmcx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Path to your folder containing the .py script
folder_path = '/content/drive/MyDrive/MCX_data'

# Add to sys.path so you can import the script
if folder_path not in sys.path:
    sys.path.append(folder_path)
from FD_msNIRS import mcx_simulation as ms
from FD_msNIRS import extract_freq as ef

In [4]:
!ls '/content/drive/MyDrive/MCX_data'

2_L_30_mu_as_df.csv	       EC.xlsx	     __pycache__
2_L_30_mu_s_primes_df.csv.csv  FD_msNIRS.py


In [5]:
pmcx.gpuinfo()

[{'name': 'NVIDIA A100-SXM4-40GB',
  'id': 1,
  'devcount': 1,
  'major': 8,
  'minor': 0,
  'globalmem': 42474471424,
  'constmem': 65536,
  'sharedmem': 49152,
  'regcount': 65536,
  'clock': 1410000,
  'sm': 108,
  'core': 6912,
  'autoblock': 64,
  'autothread': 442368,
  'maxgate': 0}]

### MCX Simulation using packgae

In [6]:

# Load the uploaded Excel files
mu_as_path = "/content/drive/MyDrive/MCX_data/2_L_30_mu_as_df.csv"
mu_s_primes_path = "/content/drive/MyDrive/MCX_data/2_L_30_mu_s_primes_df.csv.csv"

# Read the data from the files
mu_as_df = pd.read_csv(mu_as_path)
mu_s_primes_df = pd.read_csv(mu_s_primes_path)

# Display the first few rows of each to verify structure
mu_as_df.head(), mu_s_primes_df.head()

(        690       830
 0  0.177983  0.224675
 1  0.167430  0.212608
 2  0.180492  0.227544
 3  0.195039  0.244178
 4  0.165837  0.210786,
          690        830
 0  14.475115  10.631638
 1  14.383655  10.894116
 2  14.496861  10.694552
 3  14.622941  10.582863
 4  14.369844  10.783975)

In [8]:
g=0.85
wls = [690, 830]
distance = [2, 2.5, 3.0 ,3.5] # cmimport ioimport io
distance_mm = [d * 10 for d in distance]  # mm
# path to save the csv files
save_path = '/content/drive/MyDrive/running_results_4_2/'

# Iterate over each row (i.e., simulation)
for sim_idx, (ua_row, us_p_row) in enumerate(zip(mu_as_df.values, mu_s_primes_df.values)):
    ua = np.array(ua_row)
    us_prime = np.array(us_p_row)
    us = us_prime / (1 - g)

    ua_mm = ua / 10
    us_mm = us / 10
    print('ua = ', ua_mm)
    print('us = ', us_mm)
    print(f"\n--- Simulation {sim_idx + 1} ---")

    # -------- Simulation for Multi-Distances --------------
    dictt_ms = {}
    for j in range(ua_mm.shape[0]):
        print('ua = ', ua_mm[j])
        print('us = ', us_mm[j])
        TPSF_list, time_unit = ms(ua_mm[j], us_mm[j], g=0.85, n=1.370, distance=distance_mm, tend=1e-08, devf=1000, nphoton=1.2e8, source_type = 'iso')

        for i, d in enumerate(distance_mm):
            label = f"{d}, {wls[j]}"
            dictt_ms[label] = [TPSF_list[i], time_unit]

    # -------- Frequency domain analysis --------------
    target_freq = 110e6
    uac = []
    udc = []
    phase1 = []
    phase2 = []
    labels = []

    for label, (TPSF, tstep) in dictt_ms.items():
        uac_i, udc_i, p1, p2 = ef(target_freq, [TPSF], tend=1e-08, devf=1000)
        uac.append(uac_i[0])
        udc.append(udc_i[0])
        phase1.append(p1[0])
        phase2.append(p2[0])
        labels.append(label)
    # Group data by wavelength
    data_by_wl = {}

    # Reconstruct the distances and wavelengths from the labels
    for (key, (TPSF, tstep)), uac_i, udc_i, p1, p2 in zip(dictt_ms.items(), uac, udc, phase1, phase2):
        dist_str, wl_str = key.split(',')  # split key: "distance, wavelength"
        wl = float(wl_str.strip())
        d = float(dist_str.strip())

        if wl not in data_by_wl:
            data_by_wl[wl] = {'distance': [], 'uac': [], 'udc': [], 'phase': []}

        data_by_wl[wl]['distance'].append(d)
        data_by_wl[wl]['uac'].append(uac_i)
        data_by_wl[wl]['udc'].append(udc_i)
        data_by_wl[wl]['phase'].append(p1)

    # Sort the wavelengths for consistent plotting
    sorted_wls = sorted(data_by_wl.keys())


    # -------- Save Data --------------

    columns = []
    row = []

    # Get sorted lists to ensure consistent ordering
    sorted_wls = sorted(data_by_wl.keys())
    sorted_distances = sorted({d for data in data_by_wl.values() for d in data['distance']})

    for i, distance in enumerate(sorted_distances):  # i = distance index
        for j, wl in enumerate(sorted_wls):         # j = wavelength index
            data = data_by_wl[wl]
            if distance in data['distance']:
                idx = data['distance'].index(distance)

                # Use index-based naming like U_AC_1_1
                columns.extend([
                    f"U_AC_{i+1}_{j+1}",
                    f"U_DC_{i+1}_{j+1}",
                    f"Phase_{i+1}_{j+1}"
                ])

                row.extend([
                    data['uac'][idx],
                    data['udc'][idx],
                    data['phase'][idx]
                ])

    df = pd.DataFrame([row], columns=columns)
    sim_filename = f"simulation_{sim_idx + 1}.csv"
    df.to_csv(save_path + sim_filename, index=False)
    print('-------------------------    Saved Data Successfully!!!! ----------------------------')

ua =  [0.01779829 0.02246746]
us =  [9.65007661 7.08775839]

--- Simulation 1 ---
ua =  0.01779828547864764
us =  9.650076607058484
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.02246746386949256
us =  7.087758393520116
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
-------------------------    Saved Data Successfully!!!! ----------------------------
ua =  [0.01674303 0.02126078]
us =  [9.58910356 7.26274381]

--- Simulation 2 ---
ua =  0.01674303232997738
us =  9.589103561538186
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.021260784053497888
us =  7.262743805722302
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issave